In [11]:
from imagesPrepering import ImagesForModel

In [12]:
img = ImagesForModel()
img.createTriangle()
img.createSquare()

Kwadrat

In [7]:
try:
    os.mkdir("../images/preproces")
    os.mkdir("../images/preproces/square")
except:
    pass
img.copyFilesToAnotherFolder("../images/square", "../images/preproces/square", "square_")

In [41]:
img.resizeImagesInFolder("../images/preproces/square", 300, 300)
img.addObjectImagesInFolder("../images/preproces/square", "square")

Trójkąt

In [42]:
try:
    os.mkdir("../images/preproces/triangle")
except:
    pass
img.copyFilesToAnotherFolder("../images/triangle", "../images/preproces/triangle", "triangle_")

In [ ]:
img.resizeImagesInFolder("../images/preproces/triangle", 300, 300)
img.addObjectImagesInFolder("../images/preproces/triangle", "triangle")

Przenoszenie do folderu data/train

In [8]:
try:
    shutil.rmtree("../data")
    os.mkdir("../data")
    os.mkdir("../data/train")
except:
    pass
img.copyFilesToAnotherFolder("../images/preproces/square", "../data/train", "square_")
img.copyFilesToAnotherFolder("../images/preproces/triangle", "../data/train", "triangle_")

Prznoszenie losowych 20% plikow do folderu data/test

In [19]:
file_for_eval = []
for root, dirs, files in os.walk("../data/train"):
    square = list(filter(lambda x: "square" in x, files))
    random.shuffle(square)
    triangle = list(filter(lambda x: "triangle" in x, files))
    random.shuffle(triangle)
    file_for_eval += (square[:int((len(files)*0.2)/2)])
    file_for_eval += (triangle[:int((len(files)*0.2)/2)])
    random.shuffle(file_for_eval)
    print(len(file_for_eval))
    break

138


In [21]:
try:
    shutil.rmtree("../data/test")
except:
    pass

try:
    os.mkdir("../data/test")
except:
    pass

In [22]:
for file in file_for_eval:
    image = cv2.imread("../images/preproces/"+file.split('_')[0]+"/"+file)
    if np.array(image,dtype=np.float64).sum() > 1:
        cv2.imwrite("../data/test/"+file, np.array(image,dtype=np.float64))
        os.remove("../data/train/"+file)

In [32]:
img.createTriangle()
img.placeRandom("../Class_object_image/triangle.jpg", "../never_seen_images/pudzian/kozak.jpg")